In [3]:
maxsigs=2**64
F = RealField(256+100)


def pow(p,e):
    return F(p)**e
def qhitprob(qs,r):
    p = F(1/leaves)
    return binomial(qs,r)*(pow(p,r))*(pow(1-p,qs-r))

def run(l,w):
    s=l*(w-1)//2
    dp=[]
    for i in range(0,l+1):
        dp.append([])
        for j in range(0,s+1):
            dp[i].append(0)

    dp[0][0]=1
    for i in range(1,l+1):
        for j in range(0,s+1):
            for k in range(0,w):
                if k>j:
                    continue
                dp[i][j]+=dp[i-1][j-k] 
    return dp[l][s]

lenmap={}

for osec in [128]:
    lenmap[osec]={}
    lastl=100
    for w in range(8,129):
        lenmap[osec][w]=99999 
        ans = lastl
        for l in range(lastl-1,1,-1):
            if F(log(F(run(l,w)))/log2)>osec:
                ans=l
            else:
                break
        lenmap[osec][w]=ans
        lastl=ans
        print(osec,w,ans,w*ans/2.0)
print('preprocess end')

128 8 45 180.000000000000
128 9 43 193.500000000000
128 10 41 205.000000000000
128 11 39 214.500000000000
128 12 38 228.000000000000
128 13 37 240.500000000000
128 14 36 252.000000000000
128 15 35 262.500000000000
128 16 34 272.000000000000
128 17 33 280.500000000000
128 18 33 297.000000000000
128 19 32 304.000000000000
128 20 32 320.000000000000
128 21 31 325.500000000000
128 22 31 341.000000000000
128 23 30 345.000000000000
128 24 30 360.000000000000
128 25 29 362.500000000000
128 26 29 377.000000000000
128 27 29 391.500000000000
128 28 29 406.000000000000
128 29 28 406.000000000000
128 30 28 420.000000000000
128 31 28 434.000000000000
128 32 27 432.000000000000
128 33 27 445.500000000000
128 34 27 459.000000000000
128 35 27 472.500000000000
128 36 27 486.000000000000
128 37 26 481.000000000000
128 38 26 494.000000000000
128 39 26 507.000000000000
128 40 26 520.000000000000
128 41 26 533.000000000000
128 42 26 546.000000000000
128 43 25 537.500000000000
128 44 25 550.000000000000
128

In [ ]:
def size(sec,h,d,b,k,w1,hashbytes):
    l=lenmap[sec][w1]
    return ((b+1)*k+h+l*d+1)*hashbytes


def split(h,d):
    ans=[0 for i in range(d)]
    for i in range(h):
        ans[i%d]+=1
    return ans

def speed(sec,h,d,b,k,w1):
    l=lenmap[sec][w1]
    ans=k*(2**b)*(2)
    ans+=d*(2**(h/d)*(l*w1+1))
    return ans


def sec(h,d,b,k,w1):
    s=F(0)
    for r in range(100): # in fact, qhitprob(maxsigs,r)<2^{-256}*2^{-64} when r>100. Thus we can omit all r>100
        p=F(1-(1-1/F(2**b))^r)**k
        s+=qhitprob(maxsigs,r)*p
    return -F(log(F(s))/log2)

def factor(h):
    ans=[]
    for d in range(6,h):
        if h%d==0:
            ans.append(d)
    return ans


paras={}
paras[128]={}
paras[192]={}
paras[256]={}

paras[128]['small']=(63,7,12,14,16)
paras[128]['fast']=(66,22,6,33,16)

paras[192]['small']=(63,7,14,17,16)
paras[192]['fast']=(66,22,8,33,16)

paras[256]['small']=(64,8,14,22,16)
paras[256]['fast']=(68,17,9,35,16)


for osec in [128,192,256]:
    D={}
    for w in range(8,129):
        l=lenmap[osec][w]
        if not l in D:
            D[l]=w
    good_w=list(D.values())
    print(good_w)
    for ty in ['small','fast']:
        h,d,b,k,w1=paras[osec][ty]
        max_size=size(osec,h,d,b,k,w1,osec//8)
        max_hashcalls=speed(osec,h,d,b,k,w1)
        print(osec,ty,max_size,max_hashcalls)
        best_szsp=(max_size,max_hashcalls)
        best_spsz=(max_hashcalls,max_size)
        para_szsp=(h,d,b,k,w1)
        para_spsz=(h,d,b,k,w1)
        for h in [63,64,65,66,68]:
            leaves=2**h
            for b in range(3,15):
                for k in range(6,40):
                    if sec(h,0,b,k,0)>=osec-1:
                        for d in factor(h):
                            for w1 in good_w: 
                                sz=size(osec,h,d,b,k,w1,osec//8)
                                sp=speed(osec,h,d,b,k,w1)
                                if sec(h,d,b,k,w1)>=osec-1 and sz<=max_size and sp<max_hashcalls:
                                    if (sz,sp)<best_szsp:
                                        best_szsp=(sz,sp)
                                        para_szsp=(h,d,b,k,w1,lenmap[osec][w1])
                                    if (sp,sz)<best_spsz:
                                        best_spsz=(sp,sz)
                                        para_spsz=(h,d,b,k,w1,lenmap[osec][w1])
            print(h)
        print(para_szsp,best_szsp)
        print(para_spsz,best_spsz)

[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, 21, 23, 25, 29, 32, 37, 43, 51, 60, 73, 90, 114]
128 small 7744 2067968
63
64
65
66
68
(63, 9, 13, 12, 73, 22) (6880, 2047872)
(66, 11, 11, 13, 60, 23) (1025472, 7616)
128 fast 16736 100144
63
64
65
